In [1]:
from mallworld import MallWorld

In [4]:
mall_grid = MallWorld.samples["two stairs"]
task = MallWorld(maze = mall_grid, num_advs=3, 
                 rewards = {"moved":-1, "*":200, "hit-wall":-2, "runinto-people":-30, "stair":-1,"stay":-1})

In [5]:
task.reset()
task.visualize()
for i in task.adversaries_position:
    print(task.maze.unflatten_index(i))
    print(i)

#########
#......%#
#.......#
#.a#....#
#..#....#
#.......#
#.#..a..#
#%#...I.#
#########

#########
#......%#
#..######
#..#...*#
#..#..a.#
#.......#
#####...#
#%......#
#########
(1, 4, 6)
123
(0, 3, 2)
29
(0, 6, 5)
59


In [51]:
print task.perform_action_adversary_rand(3)
task.visualize()

((64, array([124,  40,  51], dtype=int64)), -1, 0)
#########
#......%#
#.......#
#..#....#
#..#a...#
#.....a.#
#.#.....#
#I#...o.#
#########

#########
#......%#
#..######
#..#...*#
#..#...a#
#.......#
#####...#
#%......#
#########
